In [956]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from time import sleep
from collections import Counter

pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set()

In [957]:
def generaPath(tipo, nombre):
    prepath = str(os.getcwd())
    #print(prepath)
    if (tipo == "I"):
        path = prepath + '\\' + nombre
    else :
        path = prepath + '\\generados\\' + nombre
    return path

In [958]:
df_cq_tr = pd.read_csv(generaPath('O', 'df_cq_tr.csv'))
df_cq_re = pd.read_csv(generaPath('O', 'df_cq_re.csv'))

df_cc_tr = pd.read_csv(generaPath('O', 'df_cc_tr.csv'))
df_cc_re = pd.read_csv(generaPath('O', 'df_cc_re.csv'))

df_ct_tr = pd.read_csv(generaPath('O', 'df_ct_tr.csv'))
df_ct_re = pd.read_csv(generaPath('O', 'df_ct_re.csv'))

df_cd_tr = pd.read_csv(generaPath('O', 'df_cd_tr.csv'))
df_cd_re = pd.read_csv(generaPath('O', 'df_cd_re.csv'))

df_cs_tr = pd.read_csv(generaPath('O', 'df_cs_tr.csv'))
df_cs_re = pd.read_csv(generaPath('O', 'df_cs_re.csv'))

df_cs_tr_sb = pd.read_csv(generaPath('O', 'df_cs_tr_sb.csv'))
df_cs_re_sb = pd.read_csv(generaPath('O', 'df_cs_re_sb.csv'))


df_sel_sb_tr = pd.read_csv(generaPath('O','df_sel_sb_tr.csv'))
df_sel_sb_re = pd.read_csv(generaPath('O','df_sel_sb_re.csv'))

df_sel_sb_re = pd.read_csv(generaPath('O','df_sel_si_tr.csv'))
df_sel_si_re = pd.read_csv(generaPath('O','df_sel_si_re.csv') )

sbs = pd.read_csv(generaPath('O','sbs.csv'))
singles = pd.read_csv(generaPath('O','singles.csv'))



In [959]:
def cantidad_no_rep():
    l= singles.sort_values("tupla")
    l2 = l.drop('R', 1)
    l3 = l2.drop_duplicates()
    return l3

In [960]:
def unsplited_tupla(secuence):
    num = str(secuence[0])
    partido = num.split(sep='|') if (num.find('|') >= 1) else num
    return partido

In [961]:
def secuencelist_unbuilder(secuence):
    num = str(secuence[0])
    partido = num.split(sep='|') if (num.find('|') >= 1) else num
    return partido

In [962]:
def crea_sorteos(single,sb, secuence):
    sorteo = []
    enteros = []
    # splitedtupla = unsplited_tupla(secuence)
    #enteros = secuencelist_unbuilder(str(secuence[0]))
    # print(splitedtupla)
    print(" Tipo Enteros")
    print(type(enteros))
    print(enteros)
    #sorteo = singles.iloc[enteros, 0]
    #print(sorteo)
    #sorteo.append(str(sb))
    return sorteo

In [963]:
def secuencelist_builder(a,b,c,d,e):
    secuencia=[]
    secuencia.append(a)
    secuencia.append(b)
    secuencia.append(c) 
    secuencia.append(d)
    secuencia.append(e)
    palabra = '|'.join([str(et) for et in secuencia])
    return palabra, secuencia


In [964]:
def crea_secuencias(totalconteo):
    secuencias = []
    secuence = []
    lsecuence =[]
    secuencia = []
    i = 0
    a = i
    b = a + 1
    c = b + 1
    d = c + 1
    e = d + 1
    while (a <= totalconteo):
        while(b <= totalconteo):
            while(c <= totalconteo):
                while(d <= totalconteo):
                    while(e <= totalconteo):
                        secuencia, secuence = secuencelist_builder(a,b,c,d,e)
                        secuencias.append(secuencia)
                        lsecuence.append(secuence)
                        e += 1
                    d += 1
                    e = d + 1
                c += 1
                d = c + 1
                e = d + 1
            b += 1
            c = b + 1
            d = c + 1
            e = d + 1
        a = a + 1
        b = a + 1
        c = b + 1
        d = c + 1 
        e = d + 1
    return secuencias, lsecuence


In [965]:
def cayo_antes(sbs, l):
    sorteos = []
    lsecuence = []  
    secuencias = []
    cant = len(l) - 1
    secuencias, lsecuence = crea_secuencias(cant)
    cantsecuence = len(lsecuence)
    print("cant:" + str(cant) + "  cantsecuence: " + str(cantsecuence))
    for index, row in sbs.iterrows():
        sb = str(row[0])
        i = 0
        for rows in secuencias:
            lsec = lsecuence[int(i)]
            sorteo = l3.iloc[lsec, 0]
            i = i + 1
            sorteos.append(sorteo.to_list())
    dft = sorteos
    # print(dft)
    return dft

In [966]:
secus = []
l3 = cantidad_no_rep()
print("valor de l3")
print(len(l3))
dft_sorteos = cayo_antes(sbs, l3)

valor de l3
15
cant:14  cantsecuence: 3003


In [967]:
def construir_trio(n1,n2,n3):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3)
    return salida

In [968]:
def construir_cuarteto(n1,n2,n3,n4):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4)
    return salida

In [969]:
def construir_quinteto(n1,n2,n3,n4,n5):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    return salida

In [970]:
def all_trios(n1,n2,n3,n4,n5):
    trio01 = construir_trio(n1,n2,n3)
    trio02 = construir_trio(n1,n2,n4)
    trio03 = construir_trio(n1,n2,n5)
    trio04 = construir_trio(n2,n3,n4)
    trio05 = construir_trio(n2,n3,n5)
    trio06 = construir_trio(n2,n4,n5)
    trio07 = construir_trio(n3,n4,n5)
    trio08 = construir_trio(n1,n3,n4)
    trio09 = construir_trio(n1,n3,n5)
    trio10 = construir_trio(n1,n4,n5)
    return trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10

In [971]:
def all_cuartetos(n1,n2,n3,n4,n5):
    cuarteto1 = construir_cuarteto(n1, n2, n3, n4)
    cuarteto2 = construir_cuarteto(n1, n2, n3, n5)
    cuarteto3 = construir_cuarteto(n1, n2, n4, n5)
    cuarteto4 = construir_cuarteto(n1, n3, n4, n5)
    cuarteto5 = construir_cuarteto(n2, n3, n4, n5)
    return cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5

In [972]:
def find_previous_coincidence(value_to_search, df):
    df_found = df.loc[df['tupla']== value_to_search]
    encontrados = 0
    total_rows = int(df_found.shape[0])
    if total_rows > 0:
        encontrados = 1
        return "Records Found: " + str(total_rows) +' - '+ str(value_to_search), encontrados
    else:
        return "No Records Found", encontrados

In [973]:
def validar_sumar_resultados_sorteos(dictionary, propiedad, dfbase, df_to_search):
    dictionary[propiedad] = propiedad
    dictionary
    mensaje, suma = find_previous_coincidence(dfbase, df_to_search)
    return suma

In [974]:
def ingresar_sorteo_validar(n1,n2,n3,n4,n5):
    quinteto = construir_quinteto(n1, n2, n3, n4, n5)
    cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5 = all_cuartetos(n1, n2, n3, n4, n5)
    trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10 = all_trios(n1, n2, n3, n4, n5)
    return quinteto, cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5, trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10

In [975]:
def validator_caller():
    i = 0
    dictionary = {}
    can_elegidos = len(dft_sorteos)
    while i < 2:
        df_actual = dft_sorteos[i]
        n1 = int(df_actual[0])
        n2 = int(df_actual[1])
        n3 = int(df_actual[2])
        n4 = int(df_actual[3])
        n5 = int(df_actual[4])
        print("n1 " + n1 )
        print("n2 " + n2 )
        totaltrios = 0
        totalcuartetos = 0
        totalquintetos = 0
        quinteto, cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5, trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10 = ingresar_sorteo_validar(n1,n2,n3,n4,n5)
        validar_sumar_resultados_sorteos(dictionary, "trios01tr", trio01, df_ct_tr)
        validar_sumar_resultados_sorteos(dictionary, "trios01re", trio01, df_ct_re) 
        validar_sumar_resultados_sorteos(dictionary, "trios02tr", trio02, df_ct_tr)
        validar_sumar_resultados_sorteos(dictionary, "trios02re", trio02, df_ct_re) 
        validar_sumar_resultados_sorteos(dictionary, "trios03tr", trio03, df_ct_tr)
        validar_sumar_resultados_sorteos(dictionary, "trios03re", trio03, df_ct_re) 
        validar_sumar_resultados_sorteos(dictionary, "trios04tr", trio04, df_ct_tr)
        validar_sumar_resultados_sorteos(dictionary, "trios04re", trio04, df_ct_re) 
        validar_sumar_resultados_sorteos(dictionary, "trios05tr", trio05, df_ct_tr)
        validar_sumar_resultados_sorteos(dictionary, "trios05re", trio05, df_ct_re)
        validar_sumar_resultados_sorteos(dictionary, "trios06tr", trio06, df_ct_tr)
        validar_sumar_resultados_sorteos(dictionary, "trios06re", trio06, df_ct_re) 
        validar_sumar_resultados_sorteos(dictionary, "trios07tr", trio07, df_ct_tr)
        validar_sumar_resultados_sorteos(dictionary, "trios07re", trio07, df_ct_re) 
        validar_sumar_resultados_sorteos(dictionary, "trios08tr", trio08, df_ct_tr)
        validar_sumar_resultados_sorteos(dictionary, "trios08re", trio08, df_ct_re) 
        validar_sumar_resultados_sorteos(dictionary, "trios09tr", trio09, df_ct_tr)
        validar_sumar_resultados_sorteos(dictionary, "trios09re", trio09, df_ct_re) 
        validar_sumar_resultados_sorteos(dictionary, "trios10tr", trio10, df_ct_tr)
        validar_sumar_resultados_sorteos(dictionary, "trios10re", trio10, df_ct_re)
        print("Cuartetos")
        validar_sumar_resultados_sorteos(dictionary, "cuarteto1tr", cuarteto1, df_cc_tr)
        validar_sumar_resultados_sorteos(dictionary, "cuarteto1re", cuarteto1, df_cc_re) 
        validar_sumar_resultados_sorteos(dictionary, "cuarteto2tr", cuarteto2, df_cc_tr)
        validar_sumar_resultados_sorteos(dictionary, "cuarteto2re", cuarteto2, df_cc_re) 
        validar_sumar_resultados_sorteos(dictionary, "cuarteto3tr", cuarteto3, df_cc_tr)
        validar_sumar_resultados_sorteos(dictionary, "cuarteto3re", cuarteto3, df_cc_re) 
        validar_sumar_resultados_sorteos(dictionary, "cuarteto4tr", cuarteto4, df_cc_tr)
        validar_sumar_resultados_sorteos(dictionary, "cuarteto4re", cuarteto4, df_cc_re) 
        validar_sumar_resultados_sorteos(dictionary, "cuarteto5tr", cuarteto5, df_cc_tr)
        validar_sumar_resultados_sorteos(dictionary, "cuarteto5re", cuarteto5, df_cc_re)
        print("Quintetos")
        validar_sumar_resultados_sorteos(dictionary, "quintetotr", quinteto,df_cq_tr)
        validar_sumar_resultados_sorteos(dictionary, "quintetore", quinteto,df_cq_re)
        print("---------")
        i = i + 1


In [976]:
def ingresar_sorteo_validar():
    n1 = int(input(f"ingresa el primer numero del sorteo: "))
    n2 = int(input(f"ingresa el segundo numero del sorteo: "))
    n3 = int(input(f"ingresa el tercer numero del sorteo: "))
    n4 = int(input(f"ingresa el cuarto numero del sorteo: "))    
    n5 = int(input(f"ingresa el quinto numero del sorteo: "))
    quinteto = construir_quinteto(n1, n2, n3, n4, n5)
    cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5 = all_cuartetos(n1, n2, n3, n4, n5)
    trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10 = all_trios(n1, n2, n3, n4, n5)
    return quinteto, cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5, trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10